### SHORT DESCRIPTION

* NCORE - numbers of core. For parallel pipeline NCORE > 1 (max NCORE = 8)
* DELETE_NOW - If True, bad reads (by filters) will be deleted after each filter
* IS_SHORT_FLANK - If True, ignore additional PRIMER and AD1 in R1
* CHAOS - If True, you're not sure that file(R1) is contained R1 and file(R2) is contained R2
* MIN_READ - Create own FIX and dbRIP file for each library (if NUM_READS >= MIN_READ)
* INSWINDOW_FIX - Delete reads if CURRENT_POS ~ [RESTRICT_SITE : IS + INSWINDOWS_FIX] (for INS_STRAND = +)
* SHIFT_RESTRICT_SITE - Mark reads if RESTRICT_SITE was found in [RESTRICT_SITE :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_PRIMER - Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* SHIFT_MISS_RE -  Mark reads if PRIMER was found in [IS :  IS + SHIFT] (for INS_STRAND = +)
* RE_HAMMING - Delete reads RE_HAMMING_CURRENT >= RE_HAMMING
* FLANK_ERRORS - Delete reads FLANK_ERRORS_CURRENT >= FLANK_ERRORS
* REPEAT - Delete reads REPEAT_CURRENT >= REPEAT
* MISS_PRIMER - Delete reads MISS_PRIMER_CURRENT <= MISS_PRIMER
* MISS_RE - Delelte reads MISS_RE_CURRENT <= MISS_RE
* TEMPLATE_SWITCH_MD - Delete reads MDMATCH >= TEMPLATE_SWITCH_MD
* WINDOW - Window for megaclustering
* MAIN_FLAN_LEN - For template switch. TARGET_READS = PRIMER + RE + MAIN_FLAN_LEN(of R1)

In [28]:
import os
print(os.getcwd())
os.chdir('/mnt/c/Users/Komar/Lab/')
print(os.getcwd())

/mnt/c/Users/Komar/Lab
/mnt/c/Users/Komar/Lab


In [113]:
#LIBRARY
import imp
import os

#PATHWAYS
INPUTDIR = '/mnt/c/Users/Komar/Lab/InPutDir/test'
OUTPUTDIR = '/mnt/c/Users/Komar/Lab/OutPutDir/test'

MAPPER = 'bowtie2' #bwa or bowtie2
BOWTIE_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bowtie2_index'
BWA_INDEX = '/mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bwa_index'

ALU_REF_LIB = '/mnt/c/Users/Komar/Lab/retroparty/Alu_replibrary_hg38.txt'  # fixed and polymorphic Alu insertions 
REF_GENOME = '/mnt/c/Users/Komar/Lab/UCSC_hg38/genome.fa'
REPEATS_REF_LIB = '/mnt/c/Users/Komar/Lab/retroparty/repeats_hg38.tabular'  # all known fixed repeats

#SYS PARAMETRS
NCORE = 4

#PREPROCESSING
PRIMER = 'GAGCCACCGCGC'
SHIFT = 5
MIST = 1
AD1 = 'GCGTGCTGCGG'
AD2 = 'AGGGCGGT'
BLEN = 9
RE = 'CCGGCC'
RESTRICT_SITE = {'AGCT': 'CT', 'TCGA': 'GA'} # {'GTAC': 'TAC', 'CTAG': 'TAG'} # {'restrict site': 'r2_start'}
IS_SHORT_FLANK = False
CHAOS = True
TRIMM_N = 0
TRIMM_POLY_N = False
POLY_N_R1 = (15, 0.8, 7) # poly_n_win_size_r1, poly_n_th_r1, poly_n_shift_r1
POLY_N_R2 = (15, 0.8, 7) # poly_n_win_size_r2, poly_n_th_r2, poly_n_shift_r2
SKIP_SHORT_READS = 50
MID_MIST_SHORT_READS = (3, 2) # (r1 ,r2) max mismatches in AD2 or PRIMER which will remove from reads
END_MIST_SHORT_READS = (6, 6) # (r1 ,r2) min length of AD2 or PRIMER at the reads ends
PLACE_OF_SEARCH_TAIL = (None, None) # (r1 ,r2)
MIN_SEQ_LEN_AFTER_TRIMM = (25, 25) # (r1 ,r2)

#SAMFILTER
FLAGS = [99, 83, 147, 163] # + [355, 339, 403, 419]

R_SITE_DISTANCE = 1000

#FILTERS
FIX_WINDOW = 1
MIN_READ = 1
MAX_DIST = 1000
MIN_DIST = 0
INSWINDOW_FIX = 40
SHIFT_RESTRICT_SITE = 3 
SHIFT_MISS_PRIMER = 27
SHIFT_MISS_RE = 9

#THRESHOLDS
RE_HAMMING = 2
FLANK_ERRORS = 5
REPEAT = 0.8
MISS_PRIMER = 4
MISS_RE = 2
TEMPLATE_SWITCH_MD = 30

#METACLUSTERING
WINDOW = 100
MAIN_FLANK_LEN = 11 # template switch?

### <font color='blue'>1st round</font> 

#### 1. Raw FASTQ files preprocessing

In [99]:
import imp
import trimmRE
imp.reload(trimmRE)

trimmRE.main(inputdir = INPUTDIR,
              outputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
              primer = PRIMER,
              ad1 = AD1,
              ad2 = AD2,
              blen = BLEN,
              shift = SHIFT,
              mist = MIST,
              restrict_site = RESTRICT_SITE,
              re_part = RE,
              is_short_flank = IS_SHORT_FLANK,
              chaos = CHAOS,
              n_core = NCORE,
              trimm_n = TRIMM_N,
              trimm_poly_N = TRIMM_POLY_N,
              poly_n_r1 = POLY_N_R1,
              poly_n_r2 = POLY_N_R2,
              skip_short_reads = SKIP_SHORT_READS,
              mid_mist_short_reads = MID_MIST_SHORT_READS,
              end_mist_short_reads = END_MIST_SHORT_READS,
              place_of_search_tail = PLACE_OF_SEARCH_TAIL,
              min_seq_len_after_trimm = MIN_SEQ_LEN_AFTER_TRIMM)

start: test_sample_1000_


test_sample_1000_:   0%|          | 0/1000 [00:00<?, ?it/s]

,fname,all,good,bad,primer,adapters,restrict_site,natural_zone,r2_start
0,test_sample_1000_,1000,0.68,0.32,0,0,64,165,0
1,total,1000,0.77,0.23,0,0,64,165,0


#### 2. Map FASTQ by BWA

In [101]:
import bwamem
imp.reload(bwamem)

if MAPPER == 'bowtie2':
    mapper_execline = 'bowtie2 -p 4 -I 25 -X 1000 --dovetail'
    refway = BOWTIE_INDEX
else:
    if MAPPER != 'bwa':
        print('BWA')
    mapper_execline = 'bwa mem -t 4'
    refway = BWA_INDEX

bwamem.main(inputdir = os.path.abspath(OUTPUTDIR) + '/preprocessing/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
            refway = refway,
            bwaline = mapper_execline)

  0%|          | 0/1 [00:00<?, ?it/s]

bowtie2 -p 4 -I 25 -X 1000 --dovetail -x /mnt/c/Users/Komar/Lab/UCSC_hg38/ref_bowtie2_index -1 /mnt/c/Users/Komar/Lab/OutPutDir/test/preprocessing/test_sample_1000__R1_good.fastq -2 /mnt/c/Users/Komar/Lab/OutPutDir/test/preprocessing/test_sample_1000__R2_good.fastq -S /mnt/c/Users/Komar/Lab/OutPutDir/test/mapping/test_sample_1000__.sam


#### 3. Filter SAM files

In [102]:
import samfilter_pysam
import gline
imp.reload(samfilter_pysam)
imp.reload(gline)

samfilter_pysam.main(inputdir = os.path.abspath(OUTPUTDIR) + '/mapping/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/table/',
                     flags = FLAGS, 
                     n_core = NCORE)

start: test_sample_1000__


test_sample_1000__:   0%|          | 0/1366 [00:00<?, ?it/s]

#### 4. Collapse reads by (chr, strand, pos)

In [103]:
import collapser
imp.reload(collapser)

collapser.main(inputdir = os.path.abspath(OUTPUTDIR) + '/table/',
               outputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
               target_re = RE,
               n_core = NCORE)

test_sample_1000__:   0%|          | 0/545 [00:00<?, ?it/s]

#### 5. Create own fix and polymorph ALUBASE

In [104]:
import exactmatch_fpALU
imp.reload(exactmatch_fpALU)

exactmatch_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                      outputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                      replibrary = ALU_REF_LIB,
                      refway = REF_GENOME,
                      restrict_site = RESTRICT_SITE,
                      max_dist = MAX_DIST,
                      min_dist = 100,
                      min_read = MIN_READ,
                      inswindow = FIX_WINDOW,
                      direction = 5,
                      n_core = NCORE)

create database:   0%|          | 0/1183143 [00:00<?, ?it/s]

(1165015, 8)
    CHR      START        END STRAND   NAME R_SITE  R_SITE_POS  IDX
0  chr1    8388315    8388618      -   AluY   AGCT     8388842    0
1  chr1   33554185   33554483      -  AluSc   AGCT    33554597    1
2  chr1   41942894   41943205      -   AluY   AGCT    41943318    2
3  chr1  150994812  150995102      +  AluSg   AGCT   150994651    3
4  chr1  192937857  192938175      +  AluJb   AGCT   192937587    4


replib:   0%|          | 0/48 [00:00<?, ?it/s]

test_sample_1000__:   0%|          | 0/545 [00:00<?, ?it/s]

#### 6. Delete fix and polymorph ALU from reads

In [105]:
import intersection_fpALU
imp.reload(intersection_fpALU)

intersection_fpALU.main(inputdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                        outputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
                        outputdir_fix = os.path.abspath(OUTPUTDIR) + '/fix_ins_table/',
                        replib_inputdir = os.path.abspath(OUTPUTDIR) + '/ematch_table/',
                        inswindow = INSWINDOW_FIX,
                        fix_ins = None,
                        n_core = NCORE)

rep: test_sample_1000__:   0%|          | 0/46 [00:00<?, ?it/s]

test_sample_1000__:   0%|          | 0/545 [00:00<?, ?it/s]

###  <font color='blue'>Filters</font> 


#### 7. Find restrict site around intergration point

In [107]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/notfpALU_table/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            refway = REF_GENOME,
            mseq = RESTRICT_SITE,
            mname = "R_SITE",
            shift = SHIFT_RESTRICT_SITE,
            n_core = NCORE)

test_sample_1000__:   0%|          | 0/85 [00:00<?, ?it/s]

#### 8. Find primer in flank

In [108]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_rsite/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            refway = REF_GENOME,
            mseq = PRIMER,
            mname = 'MISS_P_HAMMING',
            shift = SHIFT_MISS_PRIMER,
            n_core = NCORE)

test_sample_1000__:   0%|          | 0/85 [00:00<?, ?it/s]

#### 9. Find part of RE in flank

In [109]:
import misseq
imp.reload(misseq)

misseq.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_primer/',
            outputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
            refway = REF_GENOME,
            mseq = None,
            mname = 'MISS_RE_HAMMING',
            shift = SHIFT_MISS_RE,
            n_core = NCORE)

test_sample_1000__:   0%|          | 0/85 [00:00<?, ?it/s]

#### 10. Intersect with repeats

In [111]:
import intersection_replib
imp.reload(intersection_replib)

intersection_replib.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_re/',
                         outputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                         repeatway = REPEATS_REF_LIB,
                         n_core = NCORE)

replib:   0%|          | 0/24 [00:00<?, ?it/s]

test_sample_1000__:   0%|          | 0/34 [00:00<?, ?it/s]

#### 11. Apply filters

In [114]:
import trash_deleting
imp.reload(trash_deleting)

trash_deleting.main(inputdir = os.path.abspath(OUTPUTDIR) + '/filter_replib/',
                    outputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                    re_hamming = RE_HAMMING,
                    flank_errors = FLANK_ERRORS,
                    rsite = 'R_SITE',
                    repeat = REPEAT,
                    m_primer = MISS_PRIMER,
                    primer_name = 'P',
                    m_re = MISS_RE,
                    re_name = 'RE')

###  <font color='blue'>Metatable</font> 


#### 12. Metaclustering

In [ ]:
import metacluster
imp.reload(metacluster)

metacluster.main(inputdir = os.path.abspath(OUTPUTDIR) + '/pre_metatable/',
                 outputdir = os.path.abspath(OUTPUTDIR) + '/metatable_somatic/',
                 pcdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                 target_re = RE,
                 window = WINDOW,
                 blen = BLEN)

In [ ]:
import metacluster
imp.reload(metacluster)

metacluster.main(inputdir = os.path.abspath(OUTPUTDIR) + '/fix_ins_table/',
                 outputdir = os.path.abspath(OUTPUTDIR) + '/metatable_fix/',
                 pcdir = os.path.abspath(OUTPUTDIR) + '/collapse_table/',
                 target_re = RE,
                 window = WINDOW,
                 blen = BLEN)

#### 13. Template switch

In [ ]:
import template_switch
imp.reload(template_switch)

template_switch.main(inputdir = os.path.abspath(OUTPUTDIR) + '/metatable_somatic/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/result_somatic/',
                     primer = PRIMER,
                     main_flank_len = MAIN_FLANK_LEN,
                     template_switch_md = TEMPLATE_SWITCH_MD,
                     refway = BWA_INDEX,
                     bwaway = 'bwa')

In [ ]:
import template_switch
imp.reload(template_switch)

template_switch.main(inputdir = os.path.abspath(OUTPUTDIR) + '/metatable_fix/',
                     outputdir = os.path.abspath(OUTPUTDIR) + '/result_fix/',
                     primer = PRIMER,
                     main_flank_len = MAIN_FLANK_LEN,
                     template_switch_md = TEMPLATE_SWITCH_MD,
                     refway = BWA_INDEX,
                     bwaway = 'bwa')

# Result

### you can pull output table from 'result' folder in outputdir. Needed file is called 'metatable_ts.txt'